In [13]:
# Required Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the dataset
wine_data = pd.read_csv('WineQT.csv')

# Display the first few rows to understand the structure
wine_data.head()


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,Id
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,1
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,2
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,3
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,4


In [14]:
wine_data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
Id                      0
dtype: int64

In [15]:
# Preprocessing: Features and target variable
X = wine_data.drop(columns=['quality', 'Id'])  # Features
y = wine_data['quality']  # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Check the shape of the datasets
X_train_scaled.shape, X_test_scaled.shape, y_train.shape, y_test.shape


((914, 11), (229, 11), (914,), (229,))

In [16]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.regularizers import l2
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping


model = Sequential()
model.add(Dense(512, input_shape=(X_train_scaled.shape[1],), activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Dense(128, activation=LeakyReLU(alpha=0.1), kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Dense(1))  # Regression output for wine quality

# Compile the model
model.compile(optimizer=Nadam(learning_rate=0.001), loss='mean_squared_error')



C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [17]:
pip install keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Nadam
from sklearn.metrics import mean_squared_error
import numpy as np

# Define function to build and compile the model
def build_and_compile_model(neurons, dropout_rate, learning_rate):
    model = Sequential([
        Dense(neurons, input_shape=(X_train_scaled.shape[1],), activation='relu'),
        Dropout(dropout_rate),
        Dense(neurons // 2, activation='relu'),
        Dropout(dropout_rate),
        Dense(1)
    ])
    optimizer = Nadam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

# Define hyperparameters to try
param_grid = {
    'neurons': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [16, 32],
    'epochs': [100]  # Adjust this if you want more epochs for tuning
}

# Initialize variables to track the best model
best_model = None
best_params = {}
lowest_mse = float('inf')

# Loop through each combination of hyperparameters
for neurons in param_grid['neurons']:
    for dropout_rate in param_grid['dropout_rate']:
        for learning_rate in param_grid['learning_rate']:
            for batch_size in param_grid['batch_size']:
                # Build and train model with current hyperparameters
                model = build_and_compile_model(neurons, dropout_rate, learning_rate)
                model.fit(X_train_scaled, y_train, batch_size=batch_size, epochs=param_grid['epochs'][0], verbose=0)
                
                # Evaluate model
                y_pred = model.predict(X_test_scaled)
                mse = mean_squared_error(y_test, y_pred)
                
                # Check if this is the best model so far
                if mse < lowest_mse:
                    lowest_mse = mse
                    best_model = model
                    best_params = {
                        'neurons': neurons,
                        'dropout_rate': dropout_rate,
                        'learning_rate': learning_rate,
                        'batch_size': batch_size
                    }

print(f"Best Parameters: {best_params}")
print(f"Lowest Mean Squared Error: {lowest_mse}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


...

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 

In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_data=(X_test_scaled, y_test))


Epoch 1/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 6s 50ms/step - loss: 34.0303 - val_loss: 30.7153
Epoch 2/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 31.1190 - val_loss: 28.5410
Epoch 3/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 29.8597 - val_loss: 26.3301
Epoch 4/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 26.8653 - val_loss: 23.6099
Epoch 5/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 23.3251 - val_loss: 20.4519
Epoch 6/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 20.4605 - val_loss: 16.5835
Epoch 7/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 16.7787 - val_loss: 13.4743
Epoch 8/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 13.9406 - val_loss: 10.6005
Epoch 9/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 10.4474 - val_loss: 7.6442
Epoch 10/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 8.1105 - val_loss: 4.7703
Epoch 11/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 6.3853 - val_loss: 3.0945
Epoch 12/100
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s

In [ ]:
# Calculate Mean Squared Error, Mean Absolute Error, and R² Score
y_pred = model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


# Display results
print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Mean Squared Error: 0.3385849448345017
Mean Absolute Error: 0.4436031408185001
R² Score: 0.3915519118309021


In [ ]:
# Make predictions
y_pred = model.predict(X_test_scaled)

# Display the first few predictions
y_pred[:10]


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[5.186646 ],
       [5.3691397],
       [5.069113 ],
       [5.3256574],
       [6.096559 ],
       [6.4885187],
       [5.4630156],
       [5.137868 ],
       [5.685144 ],
       [5.2272134]], dtype=float32)